In [1]:
%load_ext blackcellmagic

In [2]:
from decouple import config
import json
import requests
import sys
import asyncio
from concurrent.futures import ThreadPoolExecutor
import nest_asyncio
from itertools import zip_longest
import aiohttp
from aiohttp import ClientSession

nest_asyncio.apply()

In [3]:
CLIENT_SECRET= config("CLIENT_SECRET")
CLIENT_ID = config("CLIENT_ID")
CLIENT_ACCESS_TOKEN = config("CLIENT_ACCESS_TOKEN")

In [4]:
import requests


def request_song_info(session, song_num, song_urls):
    base_url = "https://api.genius.com/songs/" + str(song_num)
    headers = {"Authorization": "Bearer " + CLIENT_ACCESS_TOKEN}
    response = requests.get(base_url, headers=headers)
    try:
        if response.json()["meta"]["status"] == 200:
            song_urls.append(
                (
                    song_num,
                    response.json()["response"]["song"]["title"],
                    response.json()["response"]["song"]["url"],
                )
            )
        else:
            pass
    except:
        pass

In [11]:
async def get_index_data_asynchronous(min_song, max_song, song_urls):
    """
    1. Establish an executor and number of workers
    2. Establish the session
    3. Establish the event loop
    4. Create the task by list comprenhensions
    5. Gather tasks.
    """
    with ClientSession as session:
        loop = asyncio.get_event_loop()
        tasks = [
            loop.run_in_executor(
                executor, request_song_info, *(session, song_num, song_urls)
            )
            for song_num in range(min_song, max_song)
        ]
        for response in await asyncio.gather(*tasks):
            pass


def execute_async_index_event_loop(min_song, max_song, song_urls):
    """
    This function does something analogous to compiling the get_data_asynchronously function,
    Then it executes loop.
    1. Call the get_data_function
    2. Get the event_loop
    3. Run the tasks (Much easier to understand in python 3.7, "ensure_future" was changed to "create_task")
    4. Edge_list and top_interactions will be passed to the next functions
    """
    future = asyncio.create_task(
        get_index_data_asynchronous(min_song, max_song, song_urls)
    )
    loop = asyncio.get_event_loop()
    loop.run_until_complete(future)
    return song_urls

In [12]:
%%time
song_urls = []
min_song=0
test_song =100
test_song_urls = execute_async_index_event_loop(min_song,test_song,song_urls)

AttributeError: __enter__

In [ ]:
len(test_song_urls)

In [ ]:
%%time
min_song = max([s[0] for s in song_urls])+1
max_song = 100000+min_song
song_urls = execute_async_index_event_loop(min_song, max_song,song_urls)